In [1]:
import io
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import text_classification, TextClassificationDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.utils import download_from_url, extract_archive, unicode_csv_reader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab

pd.set_option('display.max_colwidth', -1)

## Saving data from torchtext

In [2]:
NGRAMS = 2
DATADIR = "./data"
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
if not os.path.isdir(DATADIR):
    os.mkdir(DATADIR)
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root=DATADIR, ngrams=NGRAMS, vocab=None)

## Define the model

In [3]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## Create torchdataset from csv

In [4]:
def _csv_iterator(data_path, ngrams, yield_cls=False):
    tokenizer = get_tokenizer("basic_english")
    with io.open(data_path, encoding="utf8") as f:
        reader = unicode_csv_reader(f)
        for row in reader:
            tokens = ' '.join(row[1:])
            tokens = tokenizer(tokens)
            if yield_cls:
                yield int(row[0]) - 1, ngrams_iterator(tokens, ngrams)
            else:
                yield ngrams_iterator(tokens, ngrams)

def _create_data_from_iterator(vocab, iterator, include_unk):
    data = []
    labels = []
    with tqdm(unit_scale=0, unit='lines') as t:
        for cls, tokens in iterator:
            if include_unk:
                tokens = torch.tensor([vocab[token] for token in tokens])
            else:
                token_ids = list(filter(lambda x: x is not Vocab.UNK, [vocab[token]
                                        for token in tokens]))
                tokens = torch.tensor(token_ids)
            if len(tokens) == 0:
                logging.info('Row contains no tokens.')
            data.append((cls, tokens))
            labels.append(cls)
            t.update(1)
    return data, set(labels)

In [19]:
train_csv_path = "../data/ag_news_csv/train.csv"
test_csv_path = "../data/ag_news_csv/test.csv"
ngrams = 2

In [20]:
vocab = build_vocab_from_iterator(_csv_iterator(train_csv_path, ngrams))

120000lines [00:07, 15070.03lines/s]


In [21]:
train_iterator = _csv_iterator(train_csv_path, ngrams, yield_cls=True)
test_iterator = _csv_iterator(train_csv_path, ngrams, yield_cls=True)

In [22]:
train_data_set, labels = _create_data_from_iterator(vocab, train_iterator, include_unk=False)
test_data_set, test_labels = _create_data_from_iterator(vocab, test_iterator, include_unk=False)

120000lines [00:14, 8464.71lines/s]
120000lines [00:14, 8526.46lines/s]


## Initiate an instance

In [9]:
VOCAB_SIZE = len(vocab)
EMBED_DIM = 32
NUM_CLASS = len(labels)
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(DEVICE)

## Functions used to generate batch

In [10]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

## Define functions to train the model and evaluate results

In [11]:
def train_func(sub_train_):
    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(DEVICE), offsets.to(DEVICE), cls.to(DEVICE)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()
        
    # Adjust the learning rate
    scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(DEVICE), offsets.to(DEVICE), cls.to(DEVICE)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()
    
    return loss / len(data_), acc / len(data_)

## Split the dataset and run the model

In [12]:
N_EPOCHS = 5
min_valid_loss = float('inf')

In [14]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data_set) * 0.95)
sub_train_, sub_valid_ = random_split(train_data_set, [train_len, len(train_data_set) - train_len])

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print('Epoch: %d' %(epoch + 1), "time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1 time in 0 minutes, 24 seconds
	Loss: 0.0123(train)	|	Acc: 93.4%(train)
	Loss: 0.0000(valid)	|	Acc: 92.1%(valid)
Epoch: 2 time in 0 minutes, 24 seconds
	Loss: 0.0070(train)	|	Acc: 96.3%(train)
	Loss: 0.0001(valid)	|	Acc: 92.7%(valid)
Epoch: 3 time in 0 minutes, 24 seconds
	Loss: 0.0038(train)	|	Acc: 98.1%(train)
	Loss: 0.0001(valid)	|	Acc: 93.3%(valid)
Epoch: 4 time in 0 minutes, 24 seconds
	Loss: 0.0022(train)	|	Acc: 99.0%(train)
	Loss: 0.0000(valid)	|	Acc: 93.8%(valid)
Epoch: 5 time in 0 minutes, 25 seconds
	Loss: 0.0015(train)	|	Acc: 99.4%(train)
	Loss: 0.0000(valid)	|	Acc: 93.7%(valid)


## Evaluate the model with test dataset

In [23]:
test_loss, test_acc = test(test_data_set)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

	Loss: 0.0000(test)	|	Acc: 99.4%(test)


## Test on a random news

In [24]:
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

In [25]:
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token] for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

In [26]:
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

In [28]:
print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
